# insert_ads_campaign_full

- Name: ads_campaign_full
- Description: all campaign summary, including in store and online.
- Data modeling reference: https://docs.google.com/spreadsheets/d/1FlbiZEBue2SCAUo1WOkdtX5WbEDyDrZ9B2uiZUSiRio/edit?gid=2043216767#gid=2043216767
- Target Table: ads_campaign_full
- Source Table: dws_campaign_cost_sales_prd_d_inc
- Created by: alvinxyzhang
- Created Date: 2025-05-01
- Version: v1.0

In [30]:
from google.cloud import bigquery
import datetime
import pandas as pd

## Scheduled Task & Partition Table

In [11]:

client = bigquery.Client()

# Main excution
dataset_id = 'retail_dashboard'
target_table = 'ads_campaign_full'
campaign_id_ = 1
window_len_ = 7
full_table_id = f"{client.project}.{dataset_id}.{target_table}"



In [31]:
# Construct a BigQuery client object.
client = bigquery.Client()

table_schema = [
        bigquery.SchemaField("is_halo_effect", "STRING"),
        bigquery.SchemaField("campaign_id", "INTEGER"),
        bigquery.SchemaField("campaign_name", "STRING"),
        bigquery.SchemaField("campaign_type", "STRING"),
        bigquery.SchemaField("start_date", "DATE"),
        bigquery.SchemaField("end_date", "DATE"),
        bigquery.SchemaField("estimated_budget", "INTEGER"),
        bigquery.SchemaField("approved_budget", "INTEGER"),
        bigquery.SchemaField("real_spent_budget", "INTEGER"),
        bigquery.SchemaField("count_product", "INTEGER"),
        bigquery.SchemaField("discount_avg", "FLOAT"),
        bigquery.SchemaField("cost_in_store", "FLOAT"),
        bigquery.SchemaField("cost_flyer", "FLOAT"),
        bigquery.SchemaField("cost_community", "FLOAT"),
        bigquery.SchemaField("cost_loyalty", "FLOAT"),
        bigquery.SchemaField("cost_push", "FLOAT"),
        bigquery.SchemaField("cost_ad_total", "FLOAT"),
        bigquery.SchemaField("baseline_revenue", "FLOAT"),
        bigquery.SchemaField("baseline_cost", "FLOAT"),
        bigquery.SchemaField("campaign_revenue", "FLOAT"),
        bigquery.SchemaField("campaign_product_cost", "FLOAT"),
        bigquery.SchemaField("estimated_incremental_revenue", "FLOAT"),
        bigquery.SchemaField("incremental_revenue", "FLOAT")
    ]

insert_query = f"""
    WITH post_campaign_tbl AS (
  SELECT product_id
        , SUM(number_of_items) number_of_items_sum
        , AVG(retail_price) retail_price
        , AVG(retail_cost) retail_cost
        , SUM(total_price) total_price_sum
        , SUM(total_cost) total_cost_sum
        , AVG(retail_discount_price) retail_discount_price
        , SUM(total_discount_price) total_discount_price_sum
  FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc`
  WHERE `date` >= @post_campaign_start_date_ and `date` <= @post_campaign_end_date_
    AND purchase_type = 'instore'
    AND (campaign_id = @campaign_id_ OR campaign_id IS NULL) # exclude other campaign_id
  GROUP BY product_id
)
, baseline_tbl AS (
  SELECT product_id
      , SUM(number_of_items) number_of_items_sum
      , AVG(retail_price) retail_price
      , AVG(retail_cost) retail_cost
      , SUM(total_price) total_price_sum
      , SUM(total_cost) total_cost_sum
      , AVG(retail_discount_price) retail_discount_price
      , SUM(total_discount_price) total_discount_price_sum
  FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc`
  WHERE `date` >= @baseline_start_date_ and `date` <= @baseline_end_date_
    AND purchase_type = 'instore'
  GROUP BY product_id
)
, campaign_cost_tbl AS (
  -- guaranteed to have distinct product_id
  SELECT product_id
        , product_name
        , brand
        , category
        , IF(campaign_id IS NOT NULL, 1, 0) is_included_in_campaign
        , campaign_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , discount
        , estimated_sales_increase
  , AVG(count_product) count_product # count of product included in this campaign as planned
        , SUM(cost_in_store) cost_in_store
        , SUM(cost_flyer) cost_flyer
        , SUM(cost_community) cost_community
        , SUM(cost_loyalty) cost_loyalty
        , SUM(cost_push) cost_push
        , SUM(cost_ad_total) cost_ad_total
  FROM (
    SELECT DISTINCT `date`
        , product_id
        , product_name
        , brand
        , category
        , IF(campaign_id IS NOT NULL, 1, 0) is_included_in_campaign
        , campaign_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , discount
        , estimated_sales_increase
        , count_product # count of product included in this campaign as planned
        , cost_in_store
        , cost_flyer
        , cost_community
        , cost_loyalty
        , cost_push
        , cost_ad_total
    FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc` #
    WHERE `date` >= @post_campaign_start_date_
      AND `date` <= @post_campaign_end_date_
      # AND campaign_id = @campaign_id_ # it is already been sumed up in the last dws table for 7 days.
      # use dws_campaign_cost_sale_prd_d_inc will have Cartesian product between campaign event and sale type
      # e.g., campaign is conducted offline, however there is online sales
      # so it is crucial to get distinct of all records first, remove the cartesian product.
      )
    GROUP BY product_id
          , product_name
          , brand
          , category
          , IF(campaign_id IS NOT NULL, 1, 0)
          , campaign_id
          , campaign_name
          , campaign_type
          , start_date
          , end_date
          , estimated_budget
          , approved_budget
          , real_spent_budget
          , discount
          , estimated_sales_increase
)
, halo_effect_tbl AS (
    SELECT product_id
        , IF(in_campaign_brand_tbl.brand IS NOT NULL, 1, 0) is_brand_halo_effect
        , IF(in_campaign_cat_tbl.category IS NOT NULL, 1, 0) is_category_halo_effect
    FROM `positive-karma-457703-i3.retail_dashboard.dim_product_full` AS prd_tbl
    LEFT JOIN (
      SELECT DISTINCT brand
      FROM campaign_cost_tbl
      WHERE campaign_id IS NOT NULL
    ) AS in_campaign_brand_tbl
  ON prd_tbl.brand = in_campaign_brand_tbl.brand
  LEFT JOIN (
    SELECT DISTINCT category
    FROM campaign_cost_tbl
    WHERE campaign_id IS NOT NULL
  ) AS in_campaign_cat_tbl
  ON prd_tbl.category = in_campaign_cat_tbl.category
)
, campaign_total_tbl AS (
   SELECT campaign_cost_tbl.product_id
        , campaign_cost_tbl.product_name
        , campaign_cost_tbl.brand
        , is_brand_halo_effect
        , campaign_cost_tbl.category
        , is_category_halo_effect
        , campaign_cost_tbl.is_included_in_campaign
        , campaign_cost_tbl.campaign_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , discount
        , count_product
        , cost_in_store
        , cost_flyer
        , cost_community
        , cost_loyalty
        , cost_push
        , cost_ad_total
        , baseline_tbl.total_price_sum AS baseline_revenue
        , baseline_tbl.total_cost_sum AS baseline_cost
        , IF(campaign_id IS NOT NULL, post_campaign_tbl.total_discount_price_sum, post_campaign_tbl.total_price_sum) AS campaign_revenue # campaign use discounted sale price
        , post_campaign_tbl.total_cost_sum AS campaign_cost
        , estimated_sales_increase / 100 * baseline_tbl.total_price_sum AS estimated_incremental_revenue
        , IF(campaign_id IS NOT NULL, post_campaign_tbl.total_discount_price_sum, post_campaign_tbl.total_price_sum) - baseline_tbl.total_price_sum AS incremental_revenue
  FROM campaign_cost_tbl
  LEFT JOIN baseline_tbl
    ON campaign_cost_tbl.product_id = baseline_tbl.product_id
  LEFT JOIN post_campaign_tbl
    ON campaign_cost_tbl.product_id = post_campaign_tbl.product_id
  LEFT JOIN halo_effect_tbl
    ON campaign_cost_tbl.product_id = halo_effect_tbl.product_id
)

  SELECT '0' AS is_halo_effect
      , campaign_id
      , campaign_name
      , campaign_type
      , start_date
      , end_date
      , estimated_budget
      , approved_budget
      , real_spent_budget
      , count_product
      , AVG(discount) AS discount_avg
      , SUM(cost_in_store) AS cost_in_store
      , SUM(cost_flyer) AS cost_flyer
      , SUM(cost_community) AS cost_community
      , SUM(cost_loyalty) AS cost_loyalty
      , SUM(cost_push) AS cost_push
      , SUM(cost_ad_total) AS cost_ad_total
      , SUM(baseline_revenue) AS baseline_revenue
      , SUM(baseline_cost) AS baseline_cost
      , SUM(campaign_revenue) AS campaign_revenue
      , SUM(campaign_cost) AS campaign_product_cost
      , SUM(estimated_incremental_revenue) AS estimated_incremental_revenue
      , SUM(incremental_revenue) AS incremental_revenue
  FROM campaign_total_tbl
  WHERE is_included_in_campaign = 1
    AND campaign_id = @campaign_id_
  GROUP BY campaign_id
      , campaign_name
      , campaign_type
      , start_date
      , end_date
      , estimated_budget
      , approved_budget
      , real_spent_budget
      , count_product
UNION ALL
  SELECT is_halo_effect
        , campaign_info_tbl.campaign_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , count_product
        , discount_avg
        , cost_in_store
        , cost_flyer
        , cost_community
        , cost_loyalty
        , cost_push
        , cost_ad_total
        , baseline_revenue
        , baseline_cost
        , campaign_revenue
        , campaign_product_cost
        , estimated_incremental_revenue
        , incremental_revenue
  FROM
  (
    SELECT DISTINCT campaign_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , count_product
    FROM campaign_total_tbl
    WHERE campaign_id = @campaign_id_
  ) campaign_info_tbl
  LEFT JOIN
  (
    SELECT 'all' as is_halo_effect # all sales and all cost for each campaign
        , @campaign_id_ AS campaign_id
        , AVG(discount) AS discount_avg
        , SUM(cost_in_store) AS cost_in_store
        , SUM(cost_flyer) AS cost_flyer
        , SUM(cost_community) AS cost_community
        , SUM(cost_loyalty) AS cost_loyalty
        , SUM(cost_push) AS cost_push
        , SUM(cost_ad_total) AS cost_ad_total
        , SUM(baseline_revenue) AS baseline_revenue
        , SUM(baseline_cost) AS baseline_cost
        , SUM(campaign_revenue) AS campaign_revenue
        , SUM(campaign_cost) AS campaign_product_cost
        , SUM(estimated_incremental_revenue) AS estimated_incremental_revenue
        , SUM(incremental_revenue) AS incremental_revenue
    FROM campaign_total_tbl
    UNION ALL
    SELECT '1' as is_halo_effect # only halo effect
        , @campaign_id_ AS campaign_id
        , AVG(discount) AS discount_avg
        , SUM(cost_in_store) AS cost_in_store
        , SUM(cost_flyer) AS cost_flyer
        , SUM(cost_community) AS cost_community
        , SUM(cost_loyalty) AS cost_loyalty
        , SUM(cost_push) AS cost_push
        , SUM(cost_ad_total) AS cost_ad_total
        , SUM(baseline_revenue) AS baseline_revenue
        , SUM(baseline_cost) AS baseline_cost
        , SUM(campaign_revenue) AS campaign_revenue
        , SUM(campaign_cost) AS campaign_product_cost
        , SUM(estimated_incremental_revenue) AS estimated_incremental_revenue
        , SUM(incremental_revenue) AS incremental_revenue
    FROM campaign_total_tbl
    WHERE is_brand_halo_effect = 1
      OR is_category_halo_effect = 1
  ) campaign_sum_tbl
  on campaign_info_tbl.campaign_id = campaign_sum_tbl.campaign_id
"""


# Step 1: Check if table exists
def check_table(table_id_, full_table_id_, schema_, partition_column_):
  try:
      table = client.get_table(full_table_id_)
      print(f"    Table {table_id_} already exists.")
  except:
      print(f"    Creating partitioned table {table_id_}...")

      table = bigquery.Table(full_table_id_, schema=schema_)
      table.time_partitioning = bigquery.TimePartitioning(
          type_=bigquery.TimePartitioningType.DAY,
          field=partition_col
      )

      table = client.create_table(table)
      print(f"Created partitioned table {full_table_id_}")

# Step 2: Insert Query
def append_table(insert_query_, full_table_id_, query_parameters_):
  job_config = bigquery.QueryJobConfig(
      destination = full_table_id_,
      write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
      use_legacy_sql=False,
      query_parameters=query_parameters_
  )
  query_job_ = client.query(insert_query_, job_config=job_config)  # Make an API request.

  print('   Result:', query_job_.result())

  print('   Errors: ', query_job_.errors)

  return(query_job_)

# Step 3: Check Insert Rows
def check_insert_rows(partition_col_, full_table_id_, param_):
  query = f"""
      SELECT {partition_col_}, count(1) row_cnt
      FROM {full_table_id_}
      WHERE {partition_col_} = @campaign_id_
      GROUP BY {partition_col_}
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          # bigquery.ScalarQueryParameter("date_", "DATE", job_date_)
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", param_)
      ]
  )
  query_job = client.query_and_wait(query, job_config=job_config)

  row_cnt_df = query_job.to_dataframe()
  return(row_cnt_df)

# Main excution
dataset_id = 'retail_dashboard'
target_table = 'ads_campaign_full'
campaign_id_ = 1
window_len_ = 7
full_table_id = f"{client.project}.{dataset_id}.{target_table}"

query_parameters = [
        bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_),
        bigquery.ScalarQueryParameter("post_campaign_start_date_", "DATE", post_campaign_start_date_),
        bigquery.ScalarQueryParameter("post_campaign_end_date_", "DATE", post_campaign_end_date_),
        bigquery.ScalarQueryParameter("baseline_start_date_", "DATE", baseline_start_date_),
        bigquery.ScalarQueryParameter("baseline_end_date_", "DATE", baseline_end_date_)
    ]

parameter_query = f"""
  SELECT campaign_id # assupmtion: only one campaign happens during one period
       , start_date AS post_campaign_start_date
       , DATE_ADD(end_date, INTERVAL @window_len_ - 1 DAY) AS post_campaign_end_date # including today + the following 6 days = 1 week
       , DATE_SUB(start_date, INTERVAL campaign_length + @window_len_ DAY) AS baseline_start_date
       , DATE_SUB(start_date, INTERVAL 1 DAY) AS baseline_end_date
  FROM (
    SELECT DISTINCT campaign_id, start_date, end_date, DATE_DIFF(end_date, start_date, DAY) + 1 campaign_length
    FROM `positive-karma-457703-i3.retail_dashboard.dwd_campaign_d_inc`
    WHERE campaign_id = @campaign_id_
  ) AS t0
"""

print('- Execution campaign_id : ', campaign_id_)

print('- Read parameters :')
# Start the query, passing in the extra configuration.
query_job = client.query(parameter_query, job_config=bigquery.QueryJobConfig(query_parameters = [
        bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_)
        , bigquery.ScalarQueryParameter("window_len_", "INTEGER", window_len_)
    ]))  # Make an API request.
para_df = query_job.result().to_dataframe()

post_campaign_start_date_ = para_df.loc[0]['post_campaign_start_date']
post_campaign_end_date_ = para_df.loc[0]['post_campaign_end_date']
baseline_start_date_ = para_df.loc[0]['baseline_start_date']
baseline_end_date_ = para_df.loc[0]['baseline_end_date']
print(para_df)

# check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)
query_job = append_table(insert_query_ = insert_query, full_table_id_ = full_table_id, query_parameters_ = query_parameters)
print('- Finish writing to campaign_id', campaign_id_)
row_cnt_df = check_insert_rows(partition_col_ = "campaign_id", full_table_id_ = full_table_id, param_ = 1)
print('- Check: Total ', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

- Execution campaign_id :  1
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            1               2024-01-13             2024-01-26   

  baseline_start_date baseline_end_date  
0          2023-12-29        2024-01-12  
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7be2af061a20>
   Errors:  None
- Finish writing to campaign_id 1


IndexError: index 0 is out of bounds for axis 0 with size 0

In [32]:
row_cnt_df

,campaign_id,row_cnt


## Iterate over Date


In [33]:
# Main excution
dataset_id = 'retail_dashboard'
target_table = 'ads_campaign_full'
campaign_id_start = 1
campaign_id_end = 20
window_len_ = 7
full_table_id = f"{client.project}.{dataset_id}.{target_table}"

parameter_query = f"""
  SELECT campaign_id # assupmtion: only one campaign happens during one period
       , start_date AS post_campaign_start_date
       , DATE_ADD(end_date, INTERVAL @window_len_ - 1 DAY) AS post_campaign_end_date # including today + the following 6 days = 1 week
       , DATE_SUB(start_date, INTERVAL campaign_length + @window_len_ DAY) AS baseline_start_date
       , DATE_SUB(start_date, INTERVAL 1 DAY) AS baseline_end_date
  FROM (
    SELECT DISTINCT campaign_id, start_date, end_date, DATE_DIFF(end_date, start_date, DAY) + 1 campaign_length
    FROM `positive-karma-457703-i3.retail_dashboard.dwd_campaign_d_inc`
    WHERE campaign_id = @campaign_id_
  ) AS t0
"""

total_row_cnt_df = pd.DataFrame()

# iterate over range of dates
while (campaign_id_start <= campaign_id_end):
  campaign_id_ = campaign_id_start

  print('- Execution campaign_id : ', campaign_id_)

  print('- Read parameters :')
  # Start the query, passing in the extra configuration.
  query_job = client.query(parameter_query, job_config=bigquery.QueryJobConfig(query_parameters = [
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_)
          , bigquery.ScalarQueryParameter("window_len_", "INTEGER", window_len_)
      ]))  # Make an API request.
  para_df = query_job.result().to_dataframe()

  post_campaign_start_date_ = para_df.loc[0]['post_campaign_start_date']
  post_campaign_end_date_ = para_df.loc[0]['post_campaign_end_date']
  baseline_start_date_ = para_df.loc[0]['baseline_start_date']
  baseline_end_date_ = para_df.loc[0]['baseline_end_date']
  print(para_df)

  # check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)

  query_parameters = [
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_),
          bigquery.ScalarQueryParameter("post_campaign_start_date_", "DATE", post_campaign_start_date_),
          bigquery.ScalarQueryParameter("post_campaign_end_date_", "DATE", post_campaign_end_date_),
          bigquery.ScalarQueryParameter("baseline_start_date_", "DATE", baseline_start_date_),
          bigquery.ScalarQueryParameter("baseline_end_date_", "DATE", baseline_end_date_)
      ]

  query_job = append_table(insert_query_ = insert_query, full_table_id_ = full_table_id, query_parameters_ = query_parameters)
  print('- Finish writing to campaign_id', campaign_id_)
  row_cnt_df = check_insert_rows(partition_col_ = "campaign_id", full_table_id_ = full_table_id, param_ = 1)
  print('- Check: Total ', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

  campaign_id_start += 1


- Execution campaign_id :  1
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            1               2024-01-13             2024-01-26   

  baseline_start_date baseline_end_date  
0          2023-12-29        2024-01-12  
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7be2ae57da80>
   Errors:  None
- Finish writing to campaign_id 1
- Check: Total  3 rows.
- Execution campaign_id :  2
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            2               2024-02-05             2024-02-25   

  baseline_start_date baseline_end_date  
0          2024-01-14        2024-02-04  
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7be2ec337580>
   Errors:  None
- Finish writing to campaign_id 2
- Check: Total  3 rows.
- Execution campaign_id :  3
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            3               2024-02-29        

### Insert Summary

In [28]:
%%bigquery result
SELECT *
FROM `positive-karma-457703-i3.retail_dashboard.ads_campaign_full`

Query is running:   0%|          |

Downloading:   0%|          |

In [29]:
result

,is_halo_effect,campaign_id,campaign_name,campaign_type,start_date,end_date,estimated_budget,approved_budget,real_spent_budget,count_product,...,cost_community,cost_loyalty,cost_push,cost_ad_total,baseline_revenue,baseline_cost,campaign_revenue,campaign_product_cost,estimated_incremental_revenue,incremental_revenue
0,0,4,Sales Promotion Campaign Rest,Sales Promotion Campaign,2024-03-20,2024-04-10,107235,116224,107698,104.0,...,17806.080997,17739.874340,17956.210296,17999.0,667190.24,370434.74,4.760081e+05,380740.19,23751.972544,-191182.1791
1,all,4,Sales Promotion Campaign Rest,Sales Promotion Campaign,2024-03-20,2024-04-10,107235,116224,107698,104.0,...,17806.080997,17739.874340,17956.210296,17999.0,667190.24,370434.74,4.760081e+05,380740.19,23751.972544,-191182.1791
2,1,4,Sales Promotion Campaign Rest,Sales Promotion Campaign,2024-03-20,2024-04-10,107235,116224,107698,104.0,...,17806.080997,17739.874340,17956.210296,17999.0,667190.24,370434.74,4.760081e+05,380740.19,23751.972544,-191182.1791
3,all,3,Customer Retention Campaign Building,Customer Retention Campaign,2024-02-29,2024-03-07,89545,94959,89417,166.0,...,29809.000000,0.000000,0.000000,29716.0,570911.66,316973.75,2.743541e+05,275072.67,80955.273388,-296557.5709
4,1,3,Customer Retention Campaign Building,Customer Retention Campaign,2024-02-29,2024-03-07,89545,94959,89417,166.0,...,29809.000000,0.000000,0.000000,29716.0,570911.66,316973.75,2.743541e+05,275072.67,80955.273388,-296557.5709
5,0,3,Customer Retention Campaign Building,Customer Retention Campaign,2024-02-29,2024-03-07,89545,94959,89417,166.0,...,29809.000000,0.000000,0.000000,29716.0,570911.66,316973.75,2.743541e+05,275072.67,80955.273388,-296557.5709
6,0,5,Sales Promotion Campaign Value,Sales Promotion Campaign,2024-04-18,2024-04-25,141537,143417,141376,117.0,...,23382.593885,22713.626658,23960.393981,23472.1,386031.10,214332.36,2.047081e+05,223844.61,9805.189940,-181323.0130
7,1,5,Sales Promotion Campaign Value,Sales Promotion Campaign,2024-04-18,2024-04-25,141537,143417,141376,117.0,...,23382.593885,22713.626658,23960.393981,23472.1,386031.10,214332.36,2.047081e+05,223844.61,9805.189940,-181323.0130
8,all,5,Sales Promotion Campaign Value,Sales Promotion Campaign,2024-04-18,2024-04-25,141537,143417,141376,117.0,...,23382.593885,22713.626658,23960.393981,23472.1,386031.10,214332.36,2.047081e+05,223844.61,9805.189940,-181323.0130
9,all,7,Location-Based Campaign Social,Location-Based Campaign,2024-06-09,2024-06-23,133008,126887,133065,156.0,...,0.000000,0.000000,0.000000,44161.0,762425.66,423307.99,4.904010e+05,381701.47,42543.351828,-272024.7047
